In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [3]:
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 20)

In [4]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
#

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [9]:
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [10]:
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
db_teach_original = db_teach.copy()
db_test_original = db_test.copy()

In [404]:
# db_teach_good = Statistic.get_dt_good(db_teach_original)
# db_teach_bad = Statistic.get_dt_bad(db_teach_original)
# db_teach_good_sub = db_teach_good.sample(n=20000, random_state=123)

In [405]:
#db_teach = pd.concat([db_teach_bad, db_teach_good_sub])
#db_test = Statistic.get_dt_bad(db_test_original)

In [14]:
# db_teach = db_teach_original.copy()
# db_test = db_test_original.copy()

In [11]:
db_teach = db_teach_original.copy()
db_test = db_test_original.copy()

In [12]:
Statistic.get_table(db_teach, 'status')

Counter({'0': 427164, '1': 6261})

In [13]:
Statistic.get_table(db_test, 'status')

Counter({'0': 58107, '1': 755})

In [14]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm']
COL_FACTORS += ['is_city_resolved','is_gender_undefined']
COL_FACTORS

['bin',
 'amount',
 'bank_currency',
 'hour',
 'day_of_week',
 'longitude',
 'latitude',
 'phone_2_norm',
 'is_city_resolved',
 'is_gender_undefined']

In [15]:
train = db_teach[COL_FACTORS].copy()
test = db_test[COL_FACTORS].copy()

In [16]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [17]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

In [18]:
train.columns[train.isnull().any()]

Index([], dtype='object')

In [19]:
train.describe()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm,is_city_resolved,is_gender_undefined
count,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000,433425.000000
mean,530239.896713,86.075152,795.396403,10.202303,4.357619,-92.533259,36.902376,30.409672,0.608970,0.429638
std,14389.453792,155.484229,168.955127,7.950285,2.046192,15.696836,9.008166,25.564432,0.487982,0.495025
min,510004.000000,1.000000,4.000000,0.000000,1.000000,-170.722100,-43.533300,0.000000,0.000000,0.000000
25%,517805.000000,26.480000,840.000000,3.000000,3.000000,-97.000000,35.966600,10.000000,0.000000,0.000000
50%,527520.000000,31.890000,840.000000,7.000000,5.000000,-97.000000,38.000000,19.000000,1.000000,0.000000
75%,542432.000000,80.540000,840.000000,19.000000,6.000000,-83.009200,40.406700,50.000000,1.000000,1.000000
max,559998.000000,1500.000000,862.000000,23.000000,7.000000,172.633300,71.285000,99.000000,1.000000,1.000000


In [20]:
test.describe()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm,is_city_resolved,is_gender_undefined
count,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000
mean,529085.008393,84.763707,791.444735,10.636404,4.165778,-91.877736,38.165842,30.249499,0.670823,0.417655
std,14110.195640,132.065862,176.297117,7.945881,1.913739,16.410946,5.740971,25.408278,0.469918,0.493177
min,510250.000000,2.670000,4.000000,0.000000,1.000000,-161.800600,-43.533300,0.000000,0.000000,0.000000
25%,517805.000000,26.480000,840.000000,3.000000,3.000000,-97.000000,37.219000,10.000000,0.000000,0.000000
50%,527515.000000,47.660000,840.000000,9.000000,4.000000,-96.322000,38.000000,18.000000,1.000000,0.000000
75%,542418.000000,100.540000,840.000000,19.000000,6.000000,-81.624800,41.474900,50.000000,1.000000,1.000000
max,559996.000000,1500.000000,862.000000,23.000000,7.000000,172.633300,66.931600,99.000000,1.000000,1.000000


In [21]:
#factor normalize 
for col in COL_FACTORS:
    mean_s = train.loc[:, col].mean() 
    std_s = train.loc[:, col].std()
    train.loc[:, col] = train.loc[:, col]. apply(lambda x: (x - mean_s)/ std_s)
    test.loc[:, col] = test.loc[:, col]. apply(lambda x: (x - mean_s)/ std_s)

In [22]:
is_sample = True
if is_sample:
    n_sample = 40000
    random_state = 123
    train['status'] = db_teach_original['status']
    train_good = Statistic.get_dt_good(train)
    train_bad = Statistic.get_dt_bad(train)
    train_good_sub = train_good.sample(n=n_sample, random_state=random_state)
    train = pd.concat([train_bad, train_good_sub])
    train = train.drop('status', axis='columns')
    
    db_teach_good = Statistic.get_dt_good(db_teach_original)
    db_teach_bad = Statistic.get_dt_bad(db_teach_original)
    db_teach_good_sub = db_teach_good.sample(n=n_sample, random_state=random_state)
    db_teach = pd.concat([db_teach_bad, db_teach_good_sub])

In [23]:
train.describe()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm,is_city_resolved,is_gender_undefined
count,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000,46261.000000
mean,-0.013338,0.039019,0.019022,-0.006930,-0.016859,-0.005328,0.001464,0.010006,-0.007601,0.014214
std,1.005450,1.062524,0.968346,0.994260,1.001810,1.006527,0.977224,0.999373,1.001677,1.001927
min,-1.405814,-0.539702,-4.684063,-1.283262,-1.640911,-4.412822,-8.929195,-1.189531,-1.247937,-0.867912
25%,-0.864167,-0.383287,0.263997,-0.905918,-0.663486,-0.284563,-0.100340,-0.798362,-1.247937,-0.867912
50%,-0.189090,-0.321223,0.263997,-0.402791,0.313940,-0.284563,0.121848,-0.446310,0.801320,-0.867912
75%,0.847294,0.089558,0.263997,1.106589,0.802652,0.606674,0.376772,0.766312,0.801320,1.152187
max,2.068050,9.093687,0.394209,1.609715,1.291365,16.892994,3.333556,2.683037,0.801320,1.152187


In [24]:
test.describe()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm,is_city_resolved,is_gender_undefined
count,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000,58862.000000
mean,-0.080259,-0.008435,-0.023389,0.054602,-0.093755,0.041761,0.140258,-0.006265,0.126752,-0.024208
std,0.980593,0.849384,1.043455,0.999446,0.935269,1.045494,0.637307,0.993892,0.962984,0.996266
min,-1.389205,-0.536422,-4.684063,-1.283262,-1.640911,-4.412822,-8.929195,-1.189531,-1.247937,-0.867912
25%,-0.864167,-0.383287,0.263997,-0.905918,-0.663486,-0.284563,0.035149,-0.798362,-1.247937,-0.867912
50%,-0.189368,-0.247068,0.263997,-0.151228,-0.174773,-0.241370,0.121848,-0.485427,0.801320,-0.867912
75%,0.846321,0.093031,0.263997,1.106589,0.802652,0.694946,0.507598,0.766312,0.801320,1.152187
max,2.067911,9.093687,0.394209,1.609715,1.291365,16.892994,3.333556,2.683037,0.801320,1.152187


In [25]:
X_train = np.asarray(train)
X_test = np.asarray(test)

In [26]:
db_teach['status_good'] = np.where(db_teach.status == '1', 0, 1)
db_test['status_good'] = np.where(db_test.status == '1', 0, 1)

In [27]:
col_names = ['status_good', 'status']
y_train = db_teach[col_names].astype(float)
y_train = np.asarray(y_train)

y_test = db_test[col_names].astype(float)
y_test = np.asarray(y_test)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Passing a list of layers to the constructor
n_shape = len(COL_FACTORS)
model = Sequential([
    Dense(8, activation='relu', input_shape=(n_shape,)),
    Dense(10, activation='relu'),
    Dense(2, activation='softmax')
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 88        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 200
Trainable params: 200
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
history = model.fit(X_train, y_train,
                    batch_size= 64,
                    epochs= 30,
                    validation_split=0.2
                   )

Train on 37008 samples, validate on 9253 samples
Epoch 1/30
37008/37008 [==============================] - 1s 14us/sample - loss: 0.5118 - acc: 0.7776 - val_loss: 0.1904 - val_acc: 1.0000
Epoch 2/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4458 - acc: 0.8308 - val_loss: 0.1814 - val_acc: 1.0000
Epoch 3/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4429 - acc: 0.8308 - val_loss: 0.1878 - val_acc: 1.0000
Epoch 4/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4412 - acc: 0.8308 - val_loss: 0.1860 - val_acc: 1.0000
Epoch 5/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4399 - acc: 0.8308 - val_loss: 0.1851 - val_acc: 0.9998
Epoch 6/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4388 - acc: 0.8307 - val_loss: 0.1882 - val_acc: 0.9996
Epoch 7/30
37008/37008 [==============================] - 0s 11us/sample - loss: 0.4382 - acc: 0.8307 - val_loss: 0.1

In [31]:
result_df_amount = None
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [32]:
test_pred = model.predict_proba(test)
db_test["probability"] = test_pred[:, 1]

In [33]:
Statistic.get_table(db_test,'status')

Counter({'0': 58107, '1': 755})

In [34]:
result_df_amount = analyzer_prediction.get_table_prediction(description="wl, amount, 10 factors", result_df=result_df_amount, metric="amount")

In [35]:
analyzer_prediction.white_list = analyzer_prediction.get_empty_white_list()
result_df_amount = analyzer_prediction.get_table_prediction(description="amount 10 factors", result_df=result_df_amount, metric="amount")

In [36]:
result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,"wl, amount, 10 factors",3.520000,8.010000,9.970000,13.680000,15.030000,19.510000,21.900000,26.130000,40.220000,...,22992,23713,1640236.53,1754445.83,46261,6261,58862,755,120400.38,4.989361e+06
1,threshold amount,0.542548,0.499309,0.463483,0.427498,0.402907,0.375956,0.354018,0.320365,0.274459,...,22992,23713,1640236.53,1754445.83,46261,6261,58862,755,120400.38,4.989361e+06
2,amount 10 factors,2.910000,6.270000,8.450000,10.410000,12.530000,14.950000,16.230000,23.280000,35.620000,...,22992,23713,1640236.53,1754445.83,46261,6261,58862,755,120400.38,4.989361e+06
3,threshold amount,0.557403,0.525454,0.494744,0.466684,0.438991,0.415284,0.399293,0.352459,0.292404,...,22992,23713,1640236.53,1754445.83,46261,6261,58862,755,120400.38,4.989361e+06


In [447]:
result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,"wl, amount",5.410000,9.890000,12.030000,15.270000,16.330000,17.400000,19.560000,25.290000,42.890000,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
1,threshold amount,0.620072,0.598192,0.581985,0.569449,0.558266,0.549596,0.541324,0.517739,0.455238,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
2,amount,5.520000,9.450000,11.480000,12.470000,15.380000,15.970000,16.770000,21.540000,33.320000,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
3,threshold amount,0.623971,0.602057,0.589723,0.579635,0.570905,0.562205,0.553796,0.536269,0.485609,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06


In [429]:
result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,"wl, amount",5.940000,10.240000,11.710000,13.410000,16.750000,18.530000,20.340000,24.620000,44.910000,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
1,threshold amount,0.681902,0.646404,0.627119,0.605956,0.591949,0.578877,0.567544,0.544378,0.491051,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
2,amount,5.060000,8.090000,10.580000,11.710000,13.160000,15.080000,17.170000,21.470000,34.990000,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06
3,threshold amount,0.689101,0.656595,0.642614,0.627022,0.609787,0.599313,0.590654,0.564217,0.516273,...,22992,23713,1640236.53,1754445.83,26261,6261,58862,755,120400.38,4.989361e+06


In [397]:
result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,"wl, amount",6.550000,6.550000,9.030000,10.260000,11.880000,13.190000,14.640000,20.310000,38.200000,...,22992,23713,1640236.53,1754445.83,433425,6261,58862,755,120400.38,4.989361e+06
1,threshold amount,0.094648,0.076956,0.066534,0.057703,0.052856,0.049155,0.046138,0.039911,0.029761,...,22992,23713,1640236.53,1754445.83,433425,6261,58862,755,120400.38,4.989361e+06
2,amount,5.670000,6.550000,8.170000,8.610000,9.910000,11.530000,12.320000,16.220000,31.500000,...,22992,23713,1640236.53,1754445.83,433425,6261,58862,755,120400.38,4.989361e+06
3,threshold amount,0.100708,0.084382,0.074708,0.067292,0.062259,0.057258,0.052915,0.045443,0.033809,...,22992,23713,1640236.53,1754445.83,433425,6261,58862,755,120400.38,4.989361e+06
